# Суммаризатор видео

## 0. Устанавливаем необходимые зависимости

In [ ]:
#!pip install yt-dlp moviepy ffmpeg-python transformers[sentencepiece] faster-whisper torch

## 1. Скачиваем видео по ссылке

In [7]:
from pathlib import Path
import yt_dlp as ytdl

def download_video(url: str, out_dir: str = "downloads") -> Path:
    out_dir_p = Path(out_dir)
    out_dir_p.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        'outtmpl': str(out_dir_p / '%(title)s.%(ext)s'),
        'format': 'bestvideo+bestaudio/best',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }
    with ytdl.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        # инфа содержит 'title' и 'ext'
        filename = ydl.prepare_filename(info)
    return Path(filename)

In [2]:
download_video('https://www.youtube.com/shorts/n5M0lxa_Zys')

WindowsPath('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.webm')

## 2. Извлечение аудио

In [8]:
from moviepy import VideoFileClip
import ffmpeg

def extract_audio_to_wav(video_path: Path, out_wav: str = None, sr: int = 16000) -> Path:
    video_path = Path(video_path)
    if out_wav is None:
        out_wav = video_path.with_suffix('.wav')
    out_wav = Path(out_wav)

    clip = VideoFileClip(str(video_path))
    tmp_audio = video_path.with_suffix('.temp_audio.wav')
    clip.audio.write_audiofile(str(tmp_audio), fps=sr)
    clip.close()

    stream = ffmpeg.input(str(tmp_audio))
    stream = ffmpeg.output(stream, str(out_wav), ac=1, ar=sr)
    ffmpeg.run(stream, overwrite_output=True, quiet=True)

    try:
        os.remove(tmp_audio)
    except Exception:
        pass

    return out_wav

In [4]:
extract_audio_to_wav('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.webm')

MoviePy - Writing audio in downloads\Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.temp_audio.wav


MoviePy - Done.


WindowsPath('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.wav')

## 3. Траскрибация аудио

In [9]:
from faster_whisper import WhisperModel

def transcribe_audio_wisper(audio_path: Path, model_size: str = 'small', language: str  = 'ru', task: str = 'transcribe') -> str:
    model = WhisperModel(model_size, device='cpu', compute_type='float32')
    segments, info = model.transcribe(str(audio_path), beam_size=5, language=language, task=task)

    texts = []
    for segment in segments:
        texts.append(segment.text)
    full_text = ' '.join(texts)
    return full_text

In [10]:
text = transcribe_audio_wisper('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.wav')

In [11]:
text

' Дайте пример, красоты и математики.  Ты видел построение пятиугольника циркулем и линейкой?  Да.  Видел, да?  То есть вот этот треугольник берется 72-72-36,  проводится без сектриса, вот здесь равнобедленный,  вот здесь равнобедленный подобно исходному,  вот это равно этому, равно этому,  после чего это все обозначается за х,  возникает золотое сечение в отношении вот этих двух сторон.  Мы его строим циркулем и линейкой и строим пятиугольник.  Ну, по-моему, это божественно.  Вот честно, то есть я не знаю,  я когда показал эту тему Лебедеву,  а он говорил мне, что золотое сечение — это переоцененный объект,  вообще, оно мне вообще не нужно, это все это...  Много раз вообще это в интернете говорил из пользы всякую веселую лексику.  Вот, и вот когда мы с ним встречались,  у нас был с ним коллап,  который тоже такой довольно популярный уже под миллион просмотров,  и я могу сейчас тебе покажу, как это красиво на самом деле.  И он признал, что это действительно красиво.  То есть это божест

In [12]:
print(f"Input text length (characters): {len(text)}")

Input text length (characters): 1011


## 4. Суммаризация текста

In [13]:
def chunk_text(text: str, max_chars: int = 3000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end
    return chunks

In [16]:
from transformers import pipeline

def summarize_text(text: str, model_name: str, max_chunk_chars: int = 3000) -> str:
    summarizer = pipeline('summarization', model=model_name, truncation=True)
    chunks = chunk_text(text, max_chars=max_chunk_chars)
    summaries = []
    for i, ch in enumerate(chunks):
        res = summarizer(ch, max_length=200, min_length=30, do_sample=False)
        summaries.append(res[0]['summary_text'])

    if len(summaries) > 1:
        joined = ' '.join(summaries)
        res2 = summarizer(joined, max_length=200, min_length=50, do_sample=False)
        return res2[0]['summary_text']
    else:
        return summaries[0]

In [17]:
summarize_text(text, '2KKLabs/Lacia_sum_small_v1')

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'**Необходимость золотого сечения в отношении вот этих двух сторон заключается в том, что они строим циркулем и линейкой.** Этот треугольник берется 72-72-36, который проводится без сектриса, и здесь равнобедлен, что приводит к золотому сечению. Лебедев говорил, что золотое сечение — это переоцененный объект, поэтому ему это не нужно. Он также был популярен под миллион просмотров, и он признал, что это божественная вещь.'